In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
box = pd.read_csv('box.csv', encoding='CP949')
df=pd.read_csv('../new_m_data.csv')

C:\Users\campus4D035\AppData\Local\Temp\ipykernel_10600\1646000783.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('../new_m_data.csv')


In [4]:
box.columns

Index(['배송년월일', '대분류_착지물동량 가구/인테리어', '대분류_착지물동량 기타', '대분류_착지물동량 도서/음반',
       '대분류_착지물동량 디지털/가전', '대분류_착지물동량 생활/건강', '대분류_착지물동량 스포츠/레저',
       '대분류_착지물동량 식품', '대분류_착지물동량 출산/육아', '대분류_착지물동량 패션의류', '대분류_착지물동량 패션잡화',
       '대분류_착지물동량 화장품/미용'],
      dtype='object')

In [10]:
df = df[['구매일','배송시작일','배송완료일','주문일로부터배송완료시간', '택배이동기간']]

In [11]:
box.columns = ['배송완료일', '대분류_착지물동량 가구/인테리어', '대분류_착지물동량 기타', '대분류_착지물동량 도서/음반',
       '대분류_착지물동량 디지털/가전', '대분류_착지물동량 생활/건강', '대분류_착지물동량 스포츠/레저',
       '대분류_착지물동량 식품', '대분류_착지물동량 출산/육아', '대분류_착지물동량 패션의류', '대분류_착지물동량 패션잡화',
       '대분류_착지물동량 화장품/미용']

In [12]:
box['배송완료일'] = pd.to_datetime(box['배송완료일'])

In [13]:
df['배송완료일'] = pd.to_datetime(df['배송완료일'])

In [30]:
new_df = df.merge(box, on='배송완료일')[['배송완료일','주문일로부터배송완료시간', '택배이동기간', '대분류_착지물동량 가구/인테리어',
       '대분류_착지물동량 기타', '대분류_착지물동량 도서/음반', '대분류_착지물동량 디지털/가전',
       '대분류_착지물동량 생활/건강', '대분류_착지물동량 스포츠/레저', '대분류_착지물동량 식품',
       '대분류_착지물동량 출산/육아', '대분류_착지물동량 패션의류', '대분류_착지물동량 패션잡화',
       '대분류_착지물동량 화장품/미용']]

In [31]:
new_df.corr()['주문일로부터배송완료시간']

배송완료일                0.008675
주문일로부터배송완료시간         1.000000
택배이동기간               1.000000
대분류_착지물동량 가구/인테리어   -0.079720
대분류_착지물동량 기타        -0.087480
대분류_착지물동량 도서/음반     -0.075405
대분류_착지물동량 디지털/가전    -0.082061
대분류_착지물동량 생활/건강     -0.087708
대분류_착지물동량 스포츠/레저    -0.079658
대분류_착지물동량 식품        -0.083325
대분류_착지물동량 출산/육아     -0.084571
대분류_착지물동량 패션의류      -0.082352
대분류_착지물동량 패션잡화      -0.081323
대분류_착지물동량 화장품/미용    -0.059132
Name: 주문일로부터배송완료시간, dtype: float64

In [29]:
df.dtypes

구매일                     object
배송시작일                   object
배송완료일           datetime64[ns]
주문일로부터배송완료시간             int64
택배이동기간                   int64
weekday                  int32
dtype: object

In [22]:
df['weekday'] = df['배송완료일'].dt.weekday      # 0=월 … 6=일
new_df.assign(weekday=df['weekday']).groupby('weekday')[['주문일로부터배송완료시간'] + [c for c in new_df.columns if '대분류' in c]].corr()['주문일로부터배송완료시간']

weekday                   
0        주문일로부터배송완료시간         1.000000
         대분류_착지물동량 가구/인테리어   -0.069131
         대분류_착지물동량 기타        -0.063845
         대분류_착지물동량 도서/음반     -0.044176
         대분류_착지물동량 디지털/가전    -0.055428
                                ...   
6        대분류_착지물동량 식품         0.043421
         대분류_착지물동량 출산/육아      0.065776
         대분류_착지물동량 패션의류       0.053996
         대분류_착지물동량 패션잡화       0.062582
         대분류_착지물동량 화장품/미용    -0.000679
Name: 주문일로부터배송완료시간, Length: 84, dtype: float64

In [27]:
new_df.columns

Index(['주문일로부터배송완료시간', '택배이동기간', '대분류_착지물동량 가구/인테리어', '대분류_착지물동량 기타',
       '대분류_착지물동량 도서/음반', '대분류_착지물동량 디지털/가전', '대분류_착지물동량 생활/건강',
       '대분류_착지물동량 스포츠/레저', '대분류_착지물동량 식품', '대분류_착지물동량 출산/육아', '대분류_착지물동량 패션의류',
       '대분류_착지물동량 패션잡화', '대분류_착지물동량 화장품/미용'],
      dtype='object')

In [34]:
# 0) 날짜·리드타임·물량 집계
daily = (
    new_df.groupby('배송완료일')
          .agg(리드타임=('주문일로부터배송완료시간', 'mean'),
               **{f'{c}_mean': (c, 'mean')          # 각 대분류별 평균
                  for c in new_df.columns if '대분류' in c})
          .reset_index()
)

daily['weekday'] = daily['배송완료일'].dt.weekday
daily['물량']    = daily.filter(like='_mean').sum(axis=1)   # 모든 대분류 합계

# 1) 상관(Lag 0~3일)
for k in range(4):
    corr = (daily[['리드타임']]
              .assign(물량_lag=daily['물량'].shift(k))
              .corr()
              .iloc[0,1])
    print(f'lag {k}: {corr: .3f}')

# 2) 다중 회귀 (필수 변수만)
import statsmodels.formula.api as smf
formula = '리드타임 ~ 물량 + C(weekday) + I(물량 ** 0.5)'
fit = smf.ols(formula, data=daily).fit()
print(fit.summary())

lag 0: -0.268
lag 1: -0.076
lag 2:  0.112
lag 3:  0.121
                            OLS Regression Results                            
Dep. Variable:                   리드타임   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     6.211
Date:                Mon, 23 Jun 2025   Prob (F-statistic):           1.13e-07
Time:                        09:34:40   Log-Likelihood:                -75.122
No. Observations:                 522   AIC:                             168.2
Df Residuals:                     513   BIC:                             206.6
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

# 귀무가설 : 물량이 리드타임에 영향을 줄 것이다. -> 기각..